In [1]:
import mne
import pandas as pd
from mne.report import Report
from scipy import signal
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as pcs
colors = pcs.qualitative.Set1

In [4]:
path = r'C:\Users\Keerti\Downloads\MySolution\PlayBooks\data\1_MMN_preprocessed.set'

In [ ]:
def create_mne_raw(path, unit_conversion=1e-6,sampling_rate=250):
    """Convert DataFrame to MNE Raw object"""
    print("=== CREATING MNE OBJECT ===")
    
    extension = path.split('.')[-1]
    if extension =='csv':
        df = pd.read_csv(path)
        CHANNELS = df.columns[1:-1].to_list()
        eeg_data = df[CHANNELS].values.T * unit_conversion 
        print(CHANNELS)
            # Create MNE info object
        info = mne.create_info(
            ch_names=CHANNELS, 
            sfreq=sampling_rate, 
            ch_types=['eeg'] * len(CHANNELS)
        )
        
        # Create Raw object
        raw = mne.io.RawArray(eeg_data, info)
    elif extension in ('set','fdt'):
        raw = mne.io.read_raw_eeglab(path, preload=True)
    try:
        montage = mne.channels.make_standard_montage('standard_1020')
        raw.set_montage(montage, on_missing='ignore')
    except:
        print("Warning: Could not set electrode positions")
    return raw



def add_plotly_figure(report, fig, title):
    html_content = fig.to_html(include_plotlyjs='cdn')
    report.add_html(html_content, title)

def style_plot(fig, title, xaxis_title, yaxis_title, log_scale=False):
    fig.update_layout(title=title, xaxis_title=xaxis_title, yaxis_title=yaxis_title,
                     width=800, height=500, yaxis_type='log' if log_scale else 'linear')
    fig.update_xaxes(showgrid=True, gridcolor='lightgray', gridwidth=0.5)
    fig.update_yaxes(showgrid=True, gridcolor='lightgray', gridwidth=0.5)


raw = create_mne_raw(path)

=== CREATING MNE OBJECT ===


In [6]:
raw

<RawEEGLAB | 1_MMN_preprocessed.set, 33 x 155648 (608.0 s), ~39.2 MiB, data loaded>

In [7]:

# Initialize report
report = Report(title='EEG Data Report')
report.add_raw(raw=raw, title='Raw-Info', psd=False, butterfly=False)

# Prepare data
raw_filt = raw.copy().filter(1., 40.)
data_raw = raw.get_data() * 1e6  # µV
data_filt = raw_filt.get_data() * 1e6  # µV
data_raw_detrended = signal.detrend(data_raw, axis=1, type='constant')
times = raw.times
ch_names = raw.ch_names

# 1. Time Series Overlay
fig_ts_overlay = make_subplots(rows=len(ch_names), cols=1, subplot_titles=ch_names, 
                              shared_xaxes=False, vertical_spacing=0.02)

for i, ch_name in enumerate(ch_names):
    
    fig_ts_overlay.add_trace(go.Scatter(x=times, y=data_raw_detrended[i],
                                       mode='lines', name='Raw', line=dict(color='lightblue', width=1),
                                       showlegend=True if i == 0 else False), row=i+1, col=1)
    fig_ts_overlay.add_trace(go.Scatter(x=times, y=data_filt[i],
                                       mode='lines', name='Filtered', line=dict(color='darkblue', width=2),
                                       showlegend=True if i == 0 else False), row=i+1, col=1)

fig_ts_overlay.update_layout(height=400*len(ch_names), title='Time Series: Raw vs Filtered Overlay',
                            legend=dict(orientation="h", yanchor="bottom", y=1.02))
for i in range(len(ch_names)):
    fig_ts_overlay.update_xaxes(title_text='Time (s)', row=i+1, col=1)
    fig_ts_overlay.update_yaxes(title_text='Amplitude (µV)', row=i+1, col=1)

add_plotly_figure(report, fig_ts_overlay, 'Time Series Overlay')

# 2. PSD Overlay
psd_raw = raw.compute_psd()
psd_filt = raw_filt.compute_psd()
freqs = psd_raw.freqs
psd_raw_values = psd_raw.get_data() * 1e12  # (µV)²/Hz
psd_filt_values = psd_filt.get_data() * 1e12  # (µV)²/Hz

fig_psd_overlay = go.Figure()
for i, ch_name in enumerate(ch_names):
    color = colors[i % len(colors)]  # Same color per channel
    fig_psd_overlay.add_trace(go.Scatter(x=freqs, y=psd_raw_values[i],
                                        mode='lines', name=f'{ch_name} Raw',
                                        line=dict(color=color, width=1, dash='dot')))
    fig_psd_overlay.add_trace(go.Scatter(x=freqs, y=psd_filt_values[i],
                                        mode='lines', name=f'{ch_name} Filtered',
                                        line=dict(color=color, width=2)))
style_plot(fig_psd_overlay, 'PSD: Raw vs Filtered Overlay', 'Frequency (Hz)', 'PSD (µV²/Hz)', log_scale=True)
add_plotly_figure(report, fig_psd_overlay, 'PSD Overlay')

# 3.  Frequency Response function
filt_params = raw_filt.info.get('filter', {})
if 'coeff' in filt_params:
    filt_coeff = filt_params['coeff']
    w, h = signal.freqz(filt_coeff, fs=raw.info['sfreq'])
else:
    filt_coeff = mne.filter.create_filter(raw._data, raw.info['sfreq'], l_freq=1.0, h_freq=40.0,
                                   method='fir', fir_design='firwin')
    w, h = signal.freqz(filt_coeff, fs=raw.info['sfreq'])

magnitude = 10**(20 * np.log10(np.abs(h)) / 20)
fig_resp = go.Figure()
fig_resp.add_trace(go.Scatter(x=w, y=magnitude, mode='lines',
                             line=dict(color='blue', width=2), showlegend=False))

fig_resp.update_layout(title='Frequency Response Function ', xaxis_title='Frequency (Hz)',
                      yaxis_title='Magnitude (Linear)', xaxis=dict(range=[0, raw.info['sfreq']/2]),
                      width=800, height=500)
fig_resp.update_xaxes(showgrid=True, gridcolor='lightgray', gridwidth=0.5)
fig_resp.update_yaxes(showgrid=True, gridcolor='lightgray', gridwidth=0.5)

add_plotly_figure(report, fig_resp, 'Frequency Response Function')

# Impulse Response Function

n_samples = len(filt_coeff)
time_samples = np.arange(n_samples) / raw.info['sfreq']

fig_impulse = go.Figure()
fig_impulse.add_trace(go.Scatter(x=time_samples, y=filt_coeff, mode='lines',
                                line=dict(color='red', width=2), showlegend=False))

fig_impulse.update_layout(title='Filter Impulse Response Function', 
                         xaxis_title='Time (s)',
                         yaxis_title='Amplitude', 
                         width=800, height=500)
fig_impulse.update_xaxes(showgrid=True, gridcolor='lightgray', gridwidth=0.5)
fig_impulse.update_yaxes(showgrid=True, gridcolor='lightgray', gridwidth=0.5)

add_plotly_figure(report, fig_impulse, 'Filter Impulse Response Function')


# Save report
report.save('mne_report.html', overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Embedding : highlightjs/atom-one-dark-reasonabl

'C:\\Users\\Keerti\\Downloads\\MySolution\\PlayBooks\\mne_report.html'